In [2]:
from transformers import AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, DataCollatorWithPadding, Trainer, EarlyStoppingCallback
import evaluate
import warnings
import torch

torch.manual_seed(42)
warnings.filterwarnings("ignore")

In [3]:
model_name = "google-bert/bert-base-german-cased"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
data_files = {"train": "articlesTrain_10kGNAD_10perSegment.csv", "test": "articlesTest_10kGNAD_10perSegment.csv"}
dataset = load_dataset("csv", data_files=data_files)

def tokenize_function(set):
    return tokenizer(set["Article"], padding="max_length", truncation=True, max_length=128)
    

dataset = dataset.map(tokenize_function, batched=True)

labels = [ "Etat", "Inland", "International", "Kultur", "Panorama", "Sport", "Web", "Wirtschaft", "Wissenschaft", ]

def label_mapping(x):
    return labels.index(x)

dataset = dataset.map(lambda x: {"label": label_mapping(x["Segment"])})

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [6]:
# Initialize a BERT model for binary classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=9)
model.config.id2label = {i: label for i, label in enumerate(labels)}
model.config.label2id = {label: i for i, label in enumerate(labels)}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Freeze all layers except the classifier
for param in model.bert.parameters():
    param.requires_grad = True

In [8]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,             
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    use_mps_device=True,
    num_train_epochs=20,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch"
)

In [9]:
metric = evaluate.load("accuracy")

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,                        # Pre-trained BERT model
    args=training_args,                 # Training arguments
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,        # Efficient batching
    compute_metrics=compute_metrics     # Custom metric function
)
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))


In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.978640,0.454545
2,No log,1.753952,0.515152
3,No log,1.536484,0.585859
4,No log,1.378393,0.646465
5,No log,1.250690,0.626263
6,No log,1.266021,0.636364
7,No log,1.303836,0.616162


TrainOutput(global_step=49, training_loss=0.8562723276566486, metrics={'train_runtime': 32.4521, 'train_samples_per_second': 61.013, 'train_steps_per_second': 4.314, 'total_flos': 45586855302912.0, 'train_loss': 0.8562723276566486, 'epoch': 7.0})

In [13]:
trainer.evaluate()

{'eval_loss': 1.2506896257400513,
 'eval_accuracy': 0.6262626262626263,
 'eval_runtime': 0.6268,
 'eval_samples_per_second': 157.939,
 'eval_steps_per_second': 11.167,
 'epoch': 7.0}

In [14]:
D = dataset["test"].select([10]).remove_columns(["Segment", "Article"])
predictions = trainer.predict(D)
predicted_class = predictions.predictions.argmax(axis=-1)[0]
print(f"Predicted class: {labels[predicted_class]} (index: {predicted_class})")

Predicted class: Wirtschaft (index: 7)


In [15]:
from transformers import pipeline

# Create a text classification pipeline using the trained model
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=False,
    device="mps" if torch.backends.mps.is_available() else "cpu"
)

# Function to classify text(s) and return label names
def classify_text(texts):
    if isinstance(texts, str):
        texts = [texts]
    
    results = classifier(texts)
    return [result['label'] for result in results]

Device set to use mps


In [16]:
classifier(["Der angekündigte Umbau des Landtagsklubs schreitet zäh voran, in der Stadt ist kein logischer Nachfolger für Bürgermeister Schaden in Sicht. Er hat ein bisserl warten müssen, aber im Dezember dürfte er den Sprung in den Landtag schaffen: Tarik Mete, 28 Jahre alt und eine der wenigen echten Nachwuchshoffnungen der Salzburger SPÖ. Der rote Jungstar mit türkischen Wurzeln, der bei der Landtagswahl 2013 immerhin 1.800 Vorzugsstimmen erreichte, soll das Mandat von Nicole Solarz (34) übernehmen, die in Karenz gehen wird. Derzeit ist Mete als Assistent des Obmanns bei der Salzburger Gebietskrankenkasse beschäftigt. Landesparteiobmann und Landtagsklubobmann Walter Steidl bestätigt im STANDARD-Gespräch entsprechende Pläne. Damit sei der zweite personalpolitische Parteivorstandsbeschluss umgesetzt, sagt Steidl. Dieser habe Metes Karriere betroffen, der andere den Bezirk Lungau. Nach dem aus privaten Gründen erfolgten Rückzug des Schwarzacher Bürgermeisters Andreas Haitzer aus dem Landtag ist der 1972 geborene Bürgermeister von St. Margarethen, Gerd Brand, nachgerückt. Der von Steidl bald nach der Wahl 2013 angekündigte Umbau des Landtagsklubs schreitet mit dem Nachrücken Metes nun zwar voran, allerdings zäh. Die lange Zeit als Ablösekandidatin gehandelte zweite Landtagspräsidentin Gudrun Mosler-Törnström (59) wird wohl bis zur nächsten Wahl bleiben. Offen ist, ob die Landesgeschäftsführerin des ÖGB, Heidi Hirschbichler (56), sich früher aus dem Landtag zurückzieht. Mittelfristig ist für die Sozialdemokraten an der Salzach freilich die Frage wesentlich bedeutsamer, wer Langzeitbürgermeister Heinz Schaden nachfolgen soll. Schaden ist seit 1992 Mitglied der Stadtregierung und seit 1999 Bürgermeister. Der 61-Jährige hat wiederholt angekündigt, bei der Bürgermeister- und Gemeinderatswahl im März 2019 nicht mehr anzutreten. Eine Entscheidung über seine Nachfolge als SPÖ-"])

[{'label': 'Inland', 'score': 0.627457857131958}]

In [21]:
import pickle

# Save the classifier pipeline to a file
with open('bert_classifier_pipeline.jl', 'wb') as f:
    pickle.dump(classifier, f)

print("Pipeline saved to 'bert_classifier_pipeline.jl'")

classifier.save_pretrained("bert_classifier_pipeline")
print("Pipeline saved to 'bert_classifier pipeline'")

Pipeline saved to 'bert_classifier_pipeline.jl'
Pipeline saved to 'bert_classifier pipeline'
